In [18]:
from we_module.we import We
import pandas as pd
import time
import datetime
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix

we = We(True)

Connected to REDSHIFT_CONN
SELECT nspname from pg_catalog.pg_namespace;


In [170]:
class feed_tables():
    
    def get_posts(self, start_date='2013-01-16', end_date = datetime.datetime.now().date(), limit = 1000):

        f = open('newsfeed_posts.sql', 'r')
        sql = f.read()
        f.close()
               
        df = we.get_tbl_query(sql + ' WHERE post_created_at > \'%s\' AND post_created_at < \'%s\' LIMIT %s'  
                              % (start_date, end_date, str(limit)) )

        df['interacted_delay'] = df['interacted_at'] - df['post_created_at']

        return df
    

    def get_user_item_matrix(self, df):
            
        df['interactions'] = 1
        
        df_coo = coo_matrix((df['interactions'], (df.ix[:,0], df.ix[:,1])))
        
        return df_coo


    def get_users(self, start_date='2013-01-16', end_date = datetime.datetime.now().date(), limit = 1000):

        f = open('user_tables.sql', 'r')
        sql = f.read()
        f.close()
               
        df = we.get_tbl_query(sql + ' LIMIT %s' % (str(limit)) )
        
        df['time_as_member'] = df['user_created_at'].apply(lambda x: datetime.datetime.now() - x)
        
        return df
    

In [177]:
tables = feed_tables()
df = tables.get_users(limit = 1000000)

reconnecting to the read connection
Read connection refreshed
WITH user_group_table AS (
    SELECT
      f.follower_id,
      coalesce(listagg(g.name, ', ') WITHIN GROUP (ORDER BY f.follower_id)) AS group_list,
      coalesce(listagg(g.description, ', ') WITHIN GROUP (ORDER BY f.follower_id)) AS description_list,
      coalesce(listagg(g.blurb, ', ') WITHIN GROUP (ORDER BY f.follower_id)) AS blurb_list

    FROM mena_public.follows f
      LEFT JOIN mena_public.groups g ON (f.followable_id = g.id)
    WHERE f.follower_type = 'User' AND f.followable_type = 'Group'
    GROUP BY f.follower_id

  ),
    user_tag_table AS (
      SELECT
        tg.taggable_id,
        coalesce(listagg(t.name, ', ') WITHIN GROUP (ORDER BY tg.taggable_id)) AS tag_list,
        coalesce(listagg(tg.context, ', ') WITHIN GROUP (ORDER BY tg.taggable_id)) AS tag_context_list

      FROM mena_public.taggings tg
        LEFT JOIN mena_public.tags t ON (t.id = tg.tag_id)
        WHERE tg.taggable_type = 'User'

    

In [181]:
df.to_csv('user_object')

In [184]:
df.columns.values

array(['id', 'name', 'user_created_at', 'gender', 'kind', 'user_bio',
       'title', 'floor', 'location_id', 'home_location_id', 'websites',
       'language_preference', 'office_name', 'office_description',
       'office_started', 'region_id', 'city', 'state', 'zip', 'country',
       'active_for_anywhere', 'active', 'opened_on', 'after_hours_enabled',
       'company_uuid', 'user_tags', 'user_tag_context', 'group_list',
       'description_list', 'blurb_list', 'total_posts',
       'total_interactions', 'company_name', 'company_id', 'company_uuid',
       'company_industry', 'company_bio', 'company_tagline',
       'current_location_id', 'company_created_at', 'updated_at',
       'number_of_employees', 'company_founded_on', 'company_city',
       'company_region', 'company_country', 'company_tags',
       'company_tag_context', 'time_as_member'], dtype=object)

In [210]:
df_interact = df[(df['total_interactions'].notnull()) | (df['total_posts'].notnull())]
df_sample = df_interact.sample(n=10000)

## NLP Taggings

In [218]:
import spacy
nlp = spacy.load('en')

nlp_feat = ['id','user_bio','title','user_tags','user_tag_context','group_list','description_list',
            'blurb_list','company_industry','company_bio','company_tagline','company_tags','company_tag_context']

df_nlp = df_sample[nlp_feat]

In [224]:
df_nlp[['user_tags', 'company_tags']]

,user_tags,company_tags
183944,"Project Management, Technology, I O, Employee ...",None
132044,None,None
224001,None,None
147213,"Wine, Film, Music, comics, Food, living, Creat...",None
55793,None,None
199186,None,None
192706,None,"Business Analysis, Business Development, Seo, ..."
84833,"financial analysis for corporate, tennis, Trav...",None
30507,"Web Development, Digital Marketing, Mentorship...","Machine Learning, Data Science"
3321,"Brand Marketing, Music, Spokesperson, Learning...",None


In [ ]:
class featurize():
    
    def decay_time(self, current_column, new_column_name):
        
        
        df['time_as_member'] = df.apply(lambda x: datetime.datetime.now() - pd.to_datetime(x))
        
        print(decay)
        
        return decay
    
    def tf_idf_model(text, unigram = True, tf_idf = False):

        feature_vect = CountVectorizer(stop_words = stopwords, ngram_range=(1,2))
        X_matrix = feature_vect.fit_transform(text.values.astype('U'))

        if tf_idf == True:

            tfidf_transformer = TfidfTransformer(use_idf = True, smooth_idf=False, norm=None).fit(X_matrix)
            bi_model = tfidf_transformer.transform(X_matrix).tocsr()
        #         bi_model = hstack((labels,X_tfidf))

        else:

        ###### to normalize the term frequency matrix ######

            tf_transformer = TfidfTransformer(use_idf = False).fit(X_matrix)
            bi_model = tf_transformer.transform(X_matrix).tocsr()

        ###### -------------------------------------- ######

        return feature_vect, bi_model



## Posts Analysis

In [79]:
tables = feed_tables()
df_posts = tables.get_posts(start_date = '2016-01-01', limit = 100000)

WITH us_users AS (
    SELECT
      u.id,
      u.location_id,
      l.country,
      l.name as location_name,
      l.city as location_city

    FROM mena_public.users u
      LEFT JOIN mena_public.locations l ON (u.location_id = l.id)
    WHERE l.country = 'United States'
),
  total_table AS (
    SELECT
      p.interaction_user_id AS user,
      CASE WHEN p.photo ISNULL
        THEN 0
      ELSE 1 END            AS has_photo,
      p.post_id,
      p.post_created_at,
      p.post_content,
      p.interaction_id,
      p.interaction_content,
      p.interaction_type,
      p.interacted_at,
      p.interaction_user_id,

      u.location_id,
      u.location_name,
      u.location_city

    FROM dw.mv_post_interactions p
      LEFT JOIN us_users u ON (u.id = p.interaction_user_id)
    WHERE interaction_type IN ('Comment', 'Like') AND p.interaction_user_id IS NOT NULL

    UNION ALL

    SELECT
      p.poster_user_id AS user,
      CASE WHEN p.photo ISNULL
        THEN 0
      ELSE 1 EN

In [ ]:
df_posts.columns.values

In [78]:

df_user_item = tables.get_user_item_matrix(df_posts.loc[:,['user','post_id']])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


## --------------------------------------------LEFTOVERS --------------------------------------------

In [ ]:
df_temp = df_likes.loc[:,('user_id','action_on_id')]
l = [(tuple(x),1) for x in df_temp.to_records(index=False)]


In [ ]:
        combine users
df = df[['user', 'post_id', 'interactions']].groupby(['post_id', 'user'])      

df = df.pivot_table(index = 'user', columns = 'post_id', values = 'interactions').fillna(0)        
    


In [ ]:
def get_interactions_df(self, random=False, limit=1000):

    '''query for user likes and comments on each post

       Increase limit to 800,000+ for all interactions
    '''

    rand_string = ""

    if random==True: rand_string = "ORDER BY random()"

    df = we.get_tbl_query("\
        SELECT time_of_event, user_id, action_on_id, event_name \
        FROM dw.v_mena_db_events \
        WHERE action_on_id IS NOT NULL and \
            ((event_name = 'Like' and event_kind = 'Post') or (event_name = 'post' and event_kind = 'comment')) \
            "+ rand_string +\
            "LIMIT " + str(limit))
    df.replace(to_replace = 'post', value = 'comment')
    df['interactions'] = 1

    return df


In [ ]:
person_u = list(sort(frame.person.unique()))
thing_u = list(sort(frame.thing.unique()))

data = frame['count'].tolist()
row = frame.person.astype('category', categories=person_u).cat.codes
col = frame.thing.astype('category', categories=thing_u).cat.codes
sparse_matrix = csr_matrix((data, (row, col)), shape=(len(person_u), len(thing_u)))

sparse_matrix 
sparse_matrix.todense()

matrix([[0, 1, 0, 1],
        [1, 0, 0, 1],
        [1, 0, 1, 0]])

In [ ]:
from scipy.sparse import coo_matrix

user_list = df_temp['user_id'].values
item_list = df_temp['action_on_id'].values
likes = np.ones(len(user_list))

print(user_list)
print(len(user_list))
print(len(item_list))
print(len(likes))
# row = df_temp.user_id.astype('category', categories = user_list).cat.codes
# col = df_temp.action_on_id.astype('category', categories = item_list).cat.codes

user_item = coo_matrix((likes, (user_list,item_list)), shape=(len(user_list), len(item_list)))

user_item.todense()
                              
#         df_new = df.pivot_table(index = 'user_id', columns = 'action_on_id', values = 'likes').fillna(0)